# Chapter 8.2 - Deep Dream

In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# We not train a model, thus we need to disable all training-specific operations
K.set_learning_phase(0)

In [3]:
model = InceptionV3(weights = 'imagenet',
                    include_top = False)

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
# Dictionary mapping layers to coefficients.
# The coefficient quantifies how much the layer's activation contributes to the loss
# that will be maximized.
# Keys must correspond to names of the layers.
layer_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 3.,
    #'mixed4' : 2.,
    'mixed5' : 1.5,
}

In [6]:
# Get the symbolic outputs of each "key" layer.
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [7]:
layer_dict

{'activation_1': <keras.layers.core.Activation at 0x2ebc9e8cdd8>,
 'activation_10': <keras.layers.core.Activation at 0x2ec6ff9fc88>,
 'activation_11': <keras.layers.core.Activation at 0x2ec6ff8dbe0>,
 'activation_12': <keras.layers.core.Activation at 0x2ec701a9470>,
 'activation_13': <keras.layers.core.Activation at 0x2ec7029f400>,
 'activation_14': <keras.layers.core.Activation at 0x2ec702e0ef0>,
 'activation_15': <keras.layers.core.Activation at 0x2ec703dafd0>,
 'activation_16': <keras.layers.core.Activation at 0x2ec704a6630>,
 'activation_17': <keras.layers.core.Activation at 0x2ec705b0e80>,
 'activation_18': <keras.layers.core.Activation at 0x2ec706fbe48>,
 'activation_19': <keras.layers.core.Activation at 0x2ec708cfac8>,
 'activation_2': <keras.layers.core.Activation at 0x2ec6425d6a0>,
 'activation_20': <keras.layers.core.Activation at 0x2ec708c01d0>,
 'activation_21': <keras.layers.core.Activation at 0x2ec70abefd0>,
 'activation_22': <keras.layers.core.Activation at 0x2ec70be1c88

In [8]:
layer_contributions.keys()

dict_keys(['mixed2', 'mixed3', 'mixed5'])

In [9]:
for keras_layer in set(layer_dict.keys()).intersection(set(layer_contributions.keys())):
    print(keras_layer + ': ' + str(layer_dict[keras_layer]))
    layer = model.get_layer(name = keras_layer)
    print('\t' + str(layer.output_shape))

mixed3: <keras.layers.merge.Concatenate object at 0x000002EC71453F98>
	(None, None, None, 768)
mixed5: <keras.layers.merge.Concatenate object at 0x000002EC728BDB00>
	(None, None, None, 768)
mixed2: <keras.layers.merge.Concatenate object at 0x000002EC71038D68>
	(None, None, None, 288)


In [10]:
# Define loss as Keras variable and initialize it as zero.
loss = K.variable(0.)

In [11]:
for layer_name in layer_contributions:
    print('Layer: ' + str(layer_name))
    coeff = layer_contributions[layer_name]
    print('\tCoeff: ' + str(coeff))
    activation = layer_dict[layer_name].output
    print('\tActivation: ' + str(activation))
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    print('\tScaling: ' + str(scaling))
    # Excluding border pixels to avoid border issues
    layer_loss = coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling
    print('\tLayer loss: ' + str(layer_loss))
    loss += layer_loss

Layer: mixed2
	Coeff: 0.2
	Activation: Tensor("mixed2/concat:0", shape=(?, ?, ?, 288), dtype=float32)
	Scaling: Tensor("Prod:0", shape=(), dtype=float32)
	Layer loss: Tensor("truediv:0", shape=(), dtype=float32)
Layer: mixed3
	Coeff: 3.0
	Activation: Tensor("mixed3/concat:0", shape=(?, ?, ?, 768), dtype=float32)
	Scaling: Tensor("Prod_1:0", shape=(), dtype=float32)
	Layer loss: Tensor("truediv_1:0", shape=(), dtype=float32)
Layer: mixed5
	Coeff: 1.5
	Activation: Tensor("mixed5/concat:0", shape=(?, ?, ?, 768), dtype=float32)
	Scaling: Tensor("Prod_2:0", shape=(), dtype=float32)
	Layer loss: Tensor("truediv_2:0", shape=(), dtype=float32)


In [12]:
loss

<tf.Tensor 'add_2:0' shape=() dtype=float32>

# Gradient ascent for picture generation

In [13]:
# Tensor of shape (?, ?, ?, 3) to hold the generated image
dream = model.input

In [14]:
# Computing the gradients of the dreamed image with regard to the loss
grads = K.gradients(loss, dream)[0]

In [15]:
# Normalize gradients
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

In [16]:
# Set up function to retrieve the value
# of the loss and gradients given an input image.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

# Image preprocessing

In [17]:
import scipy
import numpy as np
from keras.preprocessing import image

In [18]:
IMAGE_PATH = './data/Chapter 8.2 - Deep Dream/image.jpg'

In [19]:
def load_image_to_array(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    return img

In [20]:
img = load_image_to_array(image_path = IMAGE_PATH)

In [21]:
img.shape

(1260, 2048, 3)

In [22]:
img = np.expand_dims(img, axis = 0)

In [23]:
img.shape

(1, 1260, 2048, 3)

In [24]:
from keras.applications.inception_v3 import preprocess_input as inceptionV3_preprocess_input

In [25]:
def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = load_image_to_array(image_path = image_path)
    img = np.expand_dims(img, axis = 0)
    img = inceptionV3_preprocess_input(img)
    return img

In [26]:
img = preprocess_image(image_path = IMAGE_PATH)

In [27]:
img.shape

(1, 1260, 2048, 3)

In [28]:
# Playing with these hyperparameters will also allow you to achieve new effects

# Gradient ascent step size
STEP = 0.01 

# Number of scales at which to run gradient ascent
num_octave = 3  

# Size ratio between scales
octave_scale = 1.4  

# Number of ascent steps per scale
ITERATIONS = 20  

In [29]:
# If our loss gets larger than 10,
# we will interrupt the gradient ascent process, to avoid ugly artifacts
MAX_LOSS = 10.

In [30]:
# Load the image into a Numpy array
img = preprocess_image(image_path = IMAGE_PATH)

In [31]:
img.shape

(1, 1260, 2048, 3)

In [32]:
# We prepare a list of shape tuples
# defining the different scales at which we will run gradient ascent
original_shape = img.shape[1:3]
original_shape

(1260, 2048)

In [33]:
successive_shapes = [original_shape]
successive_shapes

[(1260, 2048)]

In [34]:
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

In [35]:
successive_shapes

[(1260, 2048), (900, 1462), (642, 1044)]

In [36]:
# Reverse list of shapes, so that they are in increasing order
successive_shapes = successive_shapes[::-1]

In [37]:
successive_shapes

[(642, 1044), (900, 1462), (1260, 2048)]

In [38]:
def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

In [39]:
def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    # Normalization
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [40]:
import imageio

In [41]:
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imageio.imwrite(fname, pil_img)

In [42]:
# Resize the Numpy array of the image to our smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

In [43]:
for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(x = img,
                          iterations = ITERATIONS,
                          step = STEP,
                          max_loss = MAX_LOSS)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='./data/Chapter 8.2 - Deep Dream/dream_at_scale_' + str(shape) + '.png')

save_img(img, fname = './data/Chapter 8.2 - Deep Dream/final_dream.png')

Processing image shape (642, 1044)
...Loss value at 0 : 1.1246423
...Loss value at 1 : 1.4731867
...Loss value at 2 : 1.9847761
...Loss value at 3 : 2.638072
...Loss value at 4 : 3.34002
...Loss value at 5 : 4.025421
...Loss value at 6 : 4.667389
...Loss value at 7 : 5.311434
...Loss value at 8 : 5.906174
...Loss value at 9 : 6.490458
...Loss value at 10 : 7.044338
...Loss value at 11 : 7.5871515
...Loss value at 12 : 8.095409
...Loss value at 13 : 8.596305
...Loss value at 14 : 9.070893
...Loss value at 15 : 9.534389
...Loss value at 16 : 9.971168
Processing image shape (900, 1462)
...Loss value at 0 : 3.2439234
...Loss value at 1 : 4.486898
...Loss value at 2 : 5.4103613
...Loss value at 3 : 6.1980314
...Loss value at 4 : 6.914725
...Loss value at 5 : 7.5497103
...Loss value at 6 : 8.155141
...Loss value at 7 : 8.707602
...Loss value at 8 : 9.240088
...Loss value at 9 : 9.743059
Processing image shape (1260, 2048)
...Loss value at 0 : 3.2165728
...Loss value at 1 : 4.3598604
...Loss 